# An Overview of Perceptually Relevant Metrics of Audio Similarity for Potential Use as Loss In Training Neural Networks

There are several areas that need thorough exploration:
- conventional loss functions
   - equipped with some modification to improve their perceptual relevance
   - like pre-emphasis
- complex perception focused metrics
   - mostly focused on quality assessment
   - may need some modification as well
- trained models
   - least "safe" method but likely best performing *as long as the inputs are similar to training dataset*

General notes from the literature:

- Amos Tversky researched similarity, perception, and categorization from psychology point of view. He noted human perception does not satisfy the definition of a euclidean metric [[1]](#references).
- Large portion of "music similarity" research focuses on clustering music with the aim of content delivery optimization [[1]](#references).
- MFCC based distance may be helpful [[1]](#references) (not directly mentioned) and there should be a way to make mel-cepstral distance differentiable [[2]](#references).
- 

Personal comments:

- Before settling for any similarity metric we first need to decide, whether the compared signals have to be produced with the same input signal
   - A guitar player may be able to tell if two systems are similar (or the same) even when hearing two different riffs played through them
   - Metric which does not require the same input signals on both compared systems may be helpful in some cases


## Conventional Loss

- Pros
   - easy to use
   - easy to represent
   - works the same regardless of data
   - not computationally demanding
- Cons
   - perceptually irrelevant
   - relevance can only be improved to a limited degree

Conventional loss (MSE, log-cosh,...) works well for general tasks, but is completely unrelated to human perception. However it might be usable when improved with some form of pre-emphasis (A-filtering?).



## Perception Focused Metrics

- Pros
   - perceptually relevant
   - possible (though possibly difficult) to represent
- Cons
   - may not be easy to use
   - could be computationally too complex

### Notes From Literature

- Objective models for prediction of pereptual quality can be divided into two categories: parameter-based (ITU-T G.107) and signal-based [[4]](#references).
   - Since parameter-based models like ITU-T G.107 are based on modeling communication line characteristics [[4, 5]](#references), only signal-based parameters are considered in our research.
   - Signal-based models can be further divided into no-reference and full-reference [[4]](#references). *Only full-reference models are considered for our use case.*

- PEMO-Q [[3]](#references)
   - PEMO-Q was created for lossy compression evaluation [[3]](#references).
   - Attempts to answer the problem of doubts about PEAQ being a realistic and valid model of general auditory perception [[3]](#references).
   - Auditory model
      - The signals are preprocessed in a way that may not be suitable for our problem [[3]](#references).
      - After preprocessing, the signals are transformed into "internal representation" using an auditory signal processing model [[3]](#references).
         - 35-band gammatone filterbank (basilar membrane characteristics) with each band then processed individually
         - half-wave rectification and low pass filter at 1 kHz (transformation of mechanical oscillations to neural firing rates of the inner haircells)
         - absolute hearing threshold determined from assumed maximum signal input
         - sequence of five nonliear feedback loops
            - dividing element and a low-pass (RC)
            - input is divided by low passed output
         - linear modulation filterbank, the most significant difference from previous work
            - a simplified version of PEMO-Q replaces this part with modulation-low-pass version of the auditory model (less accurate but less computationally difficult)
      - Lastly, cognitive effects are modeled in post-processing stage [[3]](#references).
   - Each channel of the outputs of auditory model are then cross correlated, which (after a weighed sum) gives a perceptual quality measure called PSM [[3]](#references).
   - PEMO-Q also defines a second, more detailed (in time) measure called PSM<sub>t</sub>, which is likely not of significance for our work [[3]](#references).
   - When using the computationally less demanding version, PEMO-Q is signal dependent [[3]](#references).
   - The correlation between the PSM and subjective ratings is higher than between PSM<sub>t</sub> and subjective ratings as long as only one type of signals is studied [[3]](#references).
   - According to the paper, it should be applicable more generally than PEAQ, but is not suitable for predicting impact of linear systems [[3]](#references).

- ViSQOLAudio [[4], [6], [7]](#references)
   - ViSQOL is an objective speech quality signal-based full-reference model published in 2015 [[6]](#references) and last updated in 2020 [[7]](#references).
   - It was later modified to create a more general model, ViSQOLAudio [[4]](#references).
   - ViSQOL is based on calculating NSIM, basically SSIM applied to neurograms [[6]](#references)
   - Model is very similar to ViSQOL (below), but has a VAD for active patch selection and originally used polynomial Q to Q<sub>MOS</sub> mapping [[6], [7]](#references).
   - VISQOLAudio is VISQOL modified to evaluate lossy compression. [[4]](#references).
   - The introduction of machine learning and output of MOS [[4]](#references) may mean original VISQOL could be better for our use case.
   - According to authors of the papers, VISQOLAudio is the first free and open source audio quality metric with accuracy comparable to proprietary metrics used in the industry [[4]](#references).
   - The model has four stages: preprocessing, pairing, comparison, and similarity to quality mapping [[4]](#references).
      - Preprocessing
         - Preprocessing starts with mid channel extraction [[4]](#references). *This step is irrelevant in our case as we intend to only work with mono audio signals.*
         - Next step is alignment process, which is meant to remove subframe misalignments caused by encoder padding [[4]](#references).
         - Lastly, the "degraded" signal is scaled to match reference signal power and Gammatone spectrograms are created [[4]](#references).
      - Pairing
         - Spectrograms are segmented into patches, ordered set of subsections of the spectrograms in respect to time [[4]](#references).
         - Patch pairs with the best similarity obtained using the Neurogram Similarity Index Measure (NSIM) are then found and most similar degraded patch is selected for each reference patch [[4]](#references). The information about which patch pair yield the best result will be used in the last step [[4]](#references) *This step may be problematic when modifying the metric to suit our needs, but as we do not need the similarty measure mapped to MOS, it may be possible to skip it.*
      - Comparison
         - NSIM is calculated for each patch pair, generating a tensor (NSIM is a vector containing a score for each frequency band) of NSIM scores [[4]](#references).
         - Patch pair NSIMs selected in step 2 are averaged to obtain a single value [[4]](#references).
      - Similarity to quality mapping
         - Similarity score is fed into a support vector regression model to obtain MOS estimate [[4]](#references).

- PEAQ [[9]](#references)
   - ITU recommendation BS.1387
   - Comes in two versions, basic and advanced, where the basic should allow for a cost-efficient real time implementation and the advanced should be more accurate [[9]](#references)
   - The algorithm calculates Model Output Variables (MOVs) and estimates MOS based on them [[9]](#references)
      - MOVs is a set of extracted features (described later)
      - MOS is obtained by feeding MOVs into a neural network
   - PEAQ incorporates several other models to obtain the MOVs [[9]](#references)
      - DIX
      - NMR
      - OASE
      - PAQM
      - PERCEVAL
      - POM
   - 

- PAQM

- PERCEVAL

- POLQA and POLQA Music [[]]()

- Comparison of PEAQ x POLQA x PEMO-Q x ViSQOLAudio [[4]](#references)
   - From fig. 4 in [[4]](#references) it seems all metrics perform relatively well, but mostly show varying results depending on the used dataset.
      - ViSQOLAudio seems to be the most consistent of the tested metrics [[4]](#references).
   - PEMO-Q and ViSQOLAudio both seem to generally fit well to the subjective scores [[4]](#references).
   - PEAQ consistently gives lower score, especially in AACvOpus15 dataset [[4]](#references).
   - POLQA performance depends on the used dataset, struggling on some data and performing well on other [[4]](#references).
   - When testing the evaluation metrics without achors:
      - ViSQOLAudio 
      - PEAQ and POLQA have large variation in prediction for high quality samples [[4]](#references).
   - Testing suggests PEAQ is sensitive to sample type [[4]](#references).
   - POLQA shows poor predictions for high-bitrate samples, which is likely because POLQA being intended for evaluation of speech signals interprets music as noise [[4]](#references).
   - 


## Trained Models

- Pros
   - perceptual relevance guaranteed by training dataset
   - should be easy to use
- Cons
   - any modification requires re-training the model
   - virtually impossible to represent
   - no guarantee of performance on data not represented in dataset

### Notes from Literature

- DPAM and CDPAM [[8]](#references)
   - Neural networks trained to perform as perceptual metric [[8]](#references)
   - No information about training dataset
      - Designed for audio with 22.05 kHz sample rate and compared to speech evaluation metrics only [[8]](#references)
      - Likely only suitable for speech signals and thus unsuitable for our needs

# References
- [1] [A Large-Scale Evaluation of Acoustic and Subjective Music-Similarity Measures](https://www.jstor.org/stable/3681827)
- [2] [Embedding a Differentiable MEL-Cepstral Synthesis Filter to a Neural Speech Synthesis System](https://arxiv.org/pdf/2211.11222)
- [3] [PEMO-Q—A New Method for Objective Audio Quality Assessment Using a Model of Auditory Perception](https://ieeexplore.ieee.org/document/1709880)
- [4] [Objective Assessment of Perceptual Audio Quality Using ViSQOLAudio](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7940042)
- [5] [G.107 : The E-model: a computational model for use in transmission planning](https://www.itu.int/rec/T-REC-G.107-201506-I/en)
- [6] [ViSQOL: an objective speech quality model](https://asmp-eurasipjournals.springeropen.com/track/pdf/10.1186/s13636-015-0054-9)
- [7] [ViSQOL v3: An Open Source Production Ready Objective Speech and Audio Metric](https://arxiv.org/abs/2004.09584)
- [8] [A Differentiable Perceptual Audio Metric Learned from Just Noticeable Differences](https://arxiv.org/abs/2001.04460)
- [9] [BS.1387-2 : Method for objective measurements of perceived audio quality](https://www.itu.int/dms_pubrec/itu-r/rec/bs/R-REC-BS.1387-2-202305-I!!PDF-E.pdf)

## Sources to go through

- [a] Auditory Feature-based Perceptual Distance
  - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10925319/
  - preprint (not reviewed yet)
  - uses a CNN in calculation
- [b] Audio retrieval based on perceptual similarity
  - https://ieeexplore.ieee.org/document/7014580
- [c] Modeling Perceptual Similarity of Audio Signals for Blind Source Separation Evaluation
  - https://www.researchgate.net/publication/220848024_Modeling_Perceptual_Similarity_of_Audio_Signals_for_Blind_Source_Separation_Evaluation
- [d] A Similarity Measure for Automatic Audio Classification
  - likely not too useful
  - https://cdn.aaai.org/Symposia/Spring/1997/SS-97-03/SS97-03-001.pdf
- [e] An Objective Metric of Human Subjective Audio Quality Optimized for a Wide Range of Audio Fidelities
  - https://ieeexplore.ieee.org/abstract/document/4358089
- [f] Music Popularity: Metrics, Characteristics, and Audio-Based Prediction
  - https://ieeexplore.ieee.org/abstract/document/8327835
- [g] POLQA Music
  - POLQA meant for evaluation of music signals
  - link to be added (can't find more than short reference in one of the ViSQOL papers)
- [h] Can we still use PEAQ? A Performance Analysis of the ITU Standard for the Objective Assessment of Perceived Audio Quality
  - PEAQ x PEMO-Q x ViSQOLAudio comparison
  - https://arxiv.org/pdf/2212.01467
- [i] Perceptual Quality Assessment for Digital Audio: PEAQ-The New ITU Standard for Objective Measurement of the Perceived Audio Quality
  - https://aes2.org/publications/elibrary-page/?id=8055
- Formal listening test reccomendation
  - ITU-R BS.1116

# Help From Others

### Metody hodnoceni zvuku od Vaska


>Ahoj,
>
>tady jsou nejake linky na systemy hodnoceni kvality zvuku, ktere by se mohly dat aplikovat na Tvuj problem.
>
>https://github.com/google/visqol Tohle je system, ktery je podle clanku obstojny a asi jeden z poslednich, o kterych vim. https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7940042
>
>Tohle je starsi system, ktery ma jednodussi impementaci, takze mozna pro zacatek by byl rozumnejsi https://ieeexplore.ieee.org/document/1709880
>
>Obecne ty systemy vyuzivaji banky filtru zvanou gammatone filterbank, ktera se da najit treba tady https://amtoolbox.org/models.php Ten zbytek algoritmu by se mel taky dat najit v tom toolboxu a vyhodnoceni uz je pomoci rovnic, co se daji snadno implementovat.
>
>
>Ja sam jsem nikdy prakticky ty systemy nepouzil. Prevzal jsem rozhodovaci cast z PEMO-Q pro jeden konferencni prispevek. Tam jsem nahradil banku filtru necim, co by melo byt vernejsi funkci sluchu, ale zase je to vypocetne nesrovnale narocnejsi. Takze bych sam spise zacal od tech zavedenych signal processingovych postupu v odkazech. Kdyby jsi o funkci sluchu chtel vedet vic, pak mam v doktorske etape predmet.
>
>Vasek
